<a href="https://colab.research.google.com/github/d-tomas/data-mining/blob/main/notebooks/data_mining_4.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selección de características

## Pasos previos

In [ ]:
# Importamos las librerías de Python que necesitaremos en este notebook

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold

pd.options.mode.chained_assignment = None  # Evitamos warnings indeseados

Vamos a trabajar de nuevo con dos conjuntos de datos en formato CSV

* `pokemon.csv`: contiene 41 características de cada uno de los 802 Pokemon desde la generación 1 hasta la 7
* `ansur_ii.csv`: *Anthropometric Survey of US Army Personnel* contiene 93 medidas corporales realizadas a 6.068 adultos (4.082 hombres y 1.986 mujeres)


In [ ]:
# Obtención de los ficheros CSV con los datos

!wget https://raw.githubusercontent.com/d-tomas/data-mining/main/datasets/pokemon.csv
!wget https://raw.githubusercontent.com/d-tomas/data-mining/main/datasets/ansur_ii.csv

In [ ]:
# Cargamos los datos de Pokemon en formato CSV

data_pokemon = pd.read_csv('pokemon.csv')
data_pokemon

In [ ]:
# Cargamos los datos de ANSUR II en formato CSV

data_ansur = pd.read_csv('ansur_ii.csv')
data_ansur

## Selección por varianza

In [ ]:
# Nos quedamos con un subconjunto de las columnas y las filas para estos ejemplos

data_pokemon = data_pokemon[data_pokemon['generation'] == 1][['name', 'type1', 'type2', 'hp', 'attack', 'defense', 'speed', 'generation']]

In [ ]:
# Las columnas con poca varianza tienen poca diferencia entre observaciones y no son útiles para el análisis
# Podemos usar 'describe' para ver columnas con poca varianza

data_pokemon.describe()  # Nos interesa el mínimo, máximo y la desviación estándar

In [ ]:
# Las columnas que tengan todos los datos iguales (sin varianza) se pueden eliminar
# En este ejemplo, la columna 'generation'

data_pokemon.drop('generation', axis=1, inplace=True)  # Utilizamos 'drop' para eliminar columnas
data_pokemon

In [ ]:
# Vemos otros ejemplos, ahora con el conjunto de datos de ANSUR II
# Miramos la forma que tiene el dataset con 'shape'

data_ansur.shape

In [ ]:
# Para aplicar las técnicas siguientes nos quedamos solo con las columnas numéricas

data_ansur_num = data_ansur.select_dtypes(exclude='object')
data_ansur_num.shape

In [ ]:
# Podemos usar 'VarianceThreshold' para eliminar características en función de su varianza
# Por defecto mantienen todas las características con varianza distinta de 0

selector = VarianceThreshold(threshold=1)  # Elimina aquellas con varianza por debajo de 'threshold'
selector.fit(data_ansur_num)  # Analiza las varianzas del DataFrame
mask = selector.get_support()  # Devuelve una serie booleana con las características seleccionadas
mask

In [ ]:
# Podemos aplicar la máscara con 'loc' y descartar las columnas afectadas
# En este ejemplo solo se ha reducido una columna

data_ansur_num = data_ansur_num.loc[:, mask]
data_ansur_num.shape

In [ ]:
# Si queremos ver las columnas que se han eliminado

data_ansur_num.loc[:,~mask].columns  # DODRace = Department of Defense Race (1=whie, 2=black, 3=hispanic, ...)

In [ ]:
# Vamos a ver un nuevo ejemplo
# Seleccionamos un conjunto de características relacionadas con el tamaño del culo :-P

data_buttocks = data_ansur[['buttockdepth', 'buttockpopliteallength', 'buttockkneelength', 'buttockheight', 'buttockcircumference']]
data_buttocks

In [ ]:
# Visualizamos con un diagrama de caja el DataFrame anterior
# A mayores valores se aprecia más dispersión

plt.figure(figsize=(10, 8))
sns.boxplot(data=data_buttocks)
plt.xlabel('Característica')
plt.ylabel('Longitud (mm)')
plt.show()

In [ ]:
# La varianza no es fácil de interpretar o comparar entre características
# Deben normalizarse las varianzas antes de usarlas para seleccionar características
# Después de la normalización, la varianza en el dataset será menor (debemos reducir el umbral de varianza)

selector = VarianceThreshold(threshold=0.005)  # Hay que inspeccionar bien los datos para fijar este umbral
selector.fit(data_ansur_num/data_ansur_num.mean())  # Para normalizar valores dividimos cada columna por su valor medio
mask = selector.get_support()
data_ansur_num.loc[:, mask].shape

## Selección por valores ausentes

In [ ]:
# Otra razón para eliminar una característica es que contenga muchos valores ausentes (NaN)
# Podemos inspeccionar esta situación con 'isna'

data_pokemon.isna()

In [ ]:
# Podemos obtener el número total de valores ausentes sumando el resultado de 'isna'

data_pokemon.isna().sum()

In [ ]:
# Para tener una idea más clara de cuántos valores ausentes tenemos, podemos calcular su ratio

data_pokemon.isna().sum() / len(data_pokemon)  # Porcentaje de valores ausentes por columna

In [ ]:
# Se puede crear una máscara que elimine columnas con un porcentaje determinado de valores ausentes

mask = data_pokemon.isna().sum() / len(data_pokemon) < 0.4  # Será True si tiene menos del 40% de valores ausentes
mask

In [ ]:
# Aplicamos la máscara al conjunto de datos con 'loc'

data_pokemon = data_pokemon.loc[:, mask]  # Nos quedamos con todas las filas, y con las columnas a True
data_pokemon

## Selección mediante visualización

In [ ]:
# La matriz de dispersión proporciona una comparación uno a uno de cada variable numérica del conjunto

sns.pairplot(data=data_ansur[['Weightlbs', 'weightkg', 'Heightin', 'Gender']], diag_kind='hist', hue='Gender')
plt.show()

## Selección por correlación

In [ ]:
# Para cuantificar la correlación de manera más precisa podemos usar medidas de correlación

plt.figure(figsize=(10, 8))
sns.heatmap(data=data_ansur[['Weightlbs', 'weightkg', 'Heightin']].corr(), annot=True, linewidth=3, cmap='Blues')
plt.show()

In [ ]:
# Podemos eliminar la parte superior del mapa de calor para evitar información duplicada
# El método 'ones_like' crea una matriz con valores True y las mismas dimensiones que la matriz de correlación
# El método 'triu' (triangle upper function) pone a False todos los valores que queden por debajo de la diagonal

plt.figure(figsize=(10, 8))
mask_heatmap = np.triu(np.ones_like(data_ansur[['Weightlbs', 'weightkg', 'Heightin']].corr(), dtype=bool))
sns.heatmap(data=data_ansur[['Weightlbs', 'weightkg', 'Heightin']].corr(), annot=True, linewidth=3, cmap='Blues', mask=mask_heatmap)
plt.show()

In [ ]:
# Vemos otro ejemplo con medidas relacionadas con el tamaño del pecho
# En este caso 'chestheight' no está correlacionado con 'hipbreadthsitting', pero sí con suprasternaleheight'

plt.figure(figsize=(10, 8))
mask_heatmap = np.triu(np.ones_like(data_ansur[['earprotrusion', 'hipbreadthsitting', 'chestheight', 'suprasternaleheight', 'chestcircumference']].corr(), dtype=bool))
sns.heatmap(data=data_ansur[['earprotrusion', 'hipbreadthsitting', 'chestheight', 'suprasternaleheight', 'chestcircumference']].corr(), annot=True, linewidth=3, cmap='Blues', mask=mask_heatmap)
plt.show()

In [ ]:
# Miramos otro ejemplo con varios huesos de la región del pecho
# Tiene sentido quedarse solo con 'cervicaleheight' o 'suprasternaleheight'
# Ambos son huesos de la misma región (pecho) y es normal que tengan tamaños relacionados
# El conocimiento del dominio (médico) es importante aquí

sns.pairplot(data=data_ansur[['suprasternaleheight', 'cervicaleheight', 'chestheight']], diag_kind='hist')
plt.show()

In [ ]:
# Repetimos el análisis anterior pero usando un mapa de calor

plt.figure(figsize=(10, 8))
mask_heatmap = np.triu(np.ones_like(data_ansur[['suprasternaleheight', 'cervicaleheight', 'chestheight']].corr(), dtype=bool))
sns.heatmap(data=data_ansur[['suprasternaleheight', 'cervicaleheight', 'chestheight']].corr(), annot=True, linewidth=3, cmap='Blues', mask=mask_heatmap)
plt.show()

In [ ]:
# Se pueden filtrar variables usando un umbral de correlación
# Debes hacerlo solo si tienes confianza en que eliminar características que están muy correlacionadas no te va a hacer perder mucha información
# Miramos el valor absoluto de la correlación (abs) para descartar también correlaciones negativas altas

data_ansur_correlation = data_ansur[['suprasternaleheight', 'cervicaleheight', 'chestheight']].corr().abs()
mask = np.triu(np.ones_like(data_ansur_correlation, dtype=bool))
data_ansur_masked = data_ansur_correlation.mask(mask)  # Pone a NaN todo lo que aparezca como True en la máscara
data_ansur_masked

In [ ]:
# Podemos buscar todas las columnas que tienen algún valor de correlación por encima de un umbral
# Lo hacemos con la parte inferior de la matriz (data_ansur_masked) para evitar que se eliminen ambas características

columns_to_drop = [c for c in data_ansur_masked.columns if any(data_ansur_masked[c] > 0.95)]
data_ansur.drop(columns_to_drop, axis=1).shape

# Referencias

* [The Complete Pokemon Dataset](https://www.kaggle.com/rounakbanik/pokemon)
* [ANSUR II](https://www.kaggle.com/seshadrikolluri/ansur-ii)
* [Características de ANSUR II explicadas](http://tools.openlab.psu.edu/publicData/ANSURII-MFR.pdf)